In [2]:
import string
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

# Задание 1

In [2]:
from IPython.display import Image
from IPython.core.display import HTML

IMG_URL = 'https://i.ibb.co/47bRcVy/bow-normalized.jpg'
Image(url=IMG_URL, width=1000, height=500)

In [3]:
documents = ['я и ты',
       'ты и я',
       'я я и только я',
       'только не я',
       'он']

cv = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
documents_cv = cv.fit_transform(documents)
df = pd.DataFrame(documents_cv.toarray(), columns=cv.get_feature_names(), index=documents)
df

,и,не,он,только,ты,я
я и ты,1,0,0,0,1,1
ты и я,1,0,0,0,1,1
я я и только я,1,0,0,1,0,3
только не я,0,1,0,1,0,1
он,0,0,1,0,0,0


In [4]:
num_of_documents = df.shape[0]
idf = dict()
for column in df.columns:
    idf[column] = np.sum(df[column] >= 1)
    
print(idf)

for key in idf.keys():
    idf[key] = np.log((1 + num_of_documents) / (1 + idf[key])) + 1
    
print(idf)

{'и': 3, 'не': 1, 'он': 1, 'только': 2, 'ты': 2, 'я': 4}
{'и': 1.4054651081081644, 'не': 2.09861228866811, 'он': 2.09861228866811, 'только': 1.6931471805599454, 'ты': 1.6931471805599454, 'я': 1.1823215567939547}


In [5]:
df = df.apply(lambda x: x / x.sum(), axis=1)
print('Таблица с абсолютными частотностями:')
print(df)

Таблица с абсолютными частотностями:
                       и        не   он    только        ты         я
я и ты          0.333333  0.000000  0.0  0.000000  0.333333  0.333333
ты и я          0.333333  0.000000  0.0  0.000000  0.333333  0.333333
я я и только я  0.200000  0.000000  0.0  0.200000  0.000000  0.600000
только не я     0.000000  0.333333  0.0  0.333333  0.000000  0.333333
он              0.000000  0.000000  1.0  0.000000  0.000000  0.000000


In [6]:
for column in df.columns:
    df[column] = df[column] * idf[column]
print('Таблица с TF-IDF:')
print(df)

Таблица с TF-IDF:
                       и        не        он    только        ты         я
я и ты          0.468488  0.000000  0.000000  0.000000  0.564382  0.394107
ты и я          0.468488  0.000000  0.000000  0.000000  0.564382  0.394107
я я и только я  0.281093  0.000000  0.000000  0.338629  0.000000  0.709393
только не я     0.000000  0.699537  0.000000  0.564382  0.000000  0.394107
он              0.000000  0.000000  2.098612  0.000000  0.000000  0.000000


# Задание 2

In [3]:
df = pd.read_csv('labeled.csv')
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [8]:
# Векторизация текстов

tf_idf_vect = TfidfVectorizer()
data = tf_idf_vect.fit_transform(df['comment']) 

# Подсчет сходства 3 и 12666 текстов

print('Близость между 3-им и 12666-ым текстами корпуса равна', cosine_similarity(data[3], data[12666])[0][0])

Близость между 3-им и 12666-ым текстами корпуса равна 0.27330886297549256


In [9]:
# Поиск самых близких текстов к тексту номер 43

index_most_cosine_distances = cosine_distances(data[43], data).argsort()[0,1:4]
result_most_cosine_distances = np.sort(cosine_distances(data[43], data))[0, 1:4]

for i in range(len(index_most_cosine_distances)):
    print('Индекс текста:', index_most_cosine_distances[i])
    print('Текст:', df.iloc[index_most_cosine_distances[i]]['comment'])
    print('Значение близости:', result_most_cosine_distances[i])
    print('____________')

Индекс текста: 1986
Текст: НУ И КАКАЯ МРАЗЬ КИДАЕТ ССЫЛКИ? ОХУЕЛИ ТАМ В КРАЙ УЖЕ?

Значение близости: 0.8395601790760389
____________
Индекс текста: 1957
Текст: Че за бригада и че за махоун? Из полицейской академии?
Значение близости: 0.8821085123946333
____________
Индекс текста: 6259
Текст: Герка ебет только даунов которые игрли а него. Ибо и геймплей и сюжетто кусок говна.

Значение близости: 0.885909190769009
____________


# Задание 3

In [4]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

def preprocessing(text):
    
    text = text.lower()
    remove = string.punctuation
    remove += '«»—…#№“”'
    text = ' '.join([morph.parse((word.strip(remove)))[0].normal_form for word in text.split(' ')])
    return text

In [5]:
df['comment'] = df['comment'].apply(preprocessing) # - для обучения

# Выделяем целевую переменную 
target = df['toxic']
df = df.drop(['toxic'], axis=1)

data_for_predict = open('2ch_corpus.txt').readlines() # - для предсказания
data_for_predict = [preprocessing(text) for text in data_for_predict]

## Первый классификатор

In [6]:
# Разбиваем выборку
X_train, X_test, y_train, y_test = train_test_split(df['comment'], target, test_size=0.3, stratify=target)

# Векторизуем первым способом
tf_idf_vect = TfidfVectorizer(
    ngram_range=(1,2),
    min_df = 3,
    max_df = 0.95,
    max_features = 1000,
    token_pattern=r'(?u)\b\w+\b'
)

X_train = tf_idf_vect.fit_transform(X_train)
X_test = tf_idf_vect.transform(X_test)

## Дерево решений

In [7]:
tree_clf = DecisionTreeClassifier(criterion='entropy', max_depth=30, random_state=127, min_samples_split=2)
tree_clf.fit(X_train, y_train)
preds = tree_clf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.76      0.89      0.82      2876
         1.0       0.66      0.43      0.52      1448

    accuracy                           0.74      4324
   macro avg       0.71      0.66      0.67      4324
weighted avg       0.72      0.74      0.72      4324



In [8]:
df_predict = tf_idf_vect.transform(data_for_predict)

y_pred_proba_tree = tree_clf.predict_proba(df_predict)
max_10_index_of_tree = (-y_pred_proba_tree[:, 1]).argsort()[:10]
max_10_proba_of_tree = -np.sort(-y_pred_proba_tree[:, 1])[:10]

In [9]:
max_10_index_of_tree

array([    0, 69096, 22685, 33521, 22687, 56703, 22689, 22684, 12830,
       12827])

In [10]:
max_10_proba_of_tree

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [11]:
# Разбиваем выборку
X_train, X_test, y_train, y_test = train_test_split(df['comment'], target, test_size=0.3, stratify=target)

# Векторизуем первым способом
cv = CountVectorizer(
    ngram_range=(1,2),
    min_df = 3,
    max_df = 0.95,
    max_features = 1000,
    token_pattern=r'(?u)\b\w+\b'
)

X_train = cv.fit_transform(X_train)
X_test = cv.transform(X_test)

## Наивный Баейс

In [12]:
nb_clf = MultinomialNB(alpha=1.5, fit_prior=False)
nb_clf.fit(X_train, y_train)
preds = nb_clf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.87      0.84      0.85      2876
         1.0       0.70      0.75      0.72      1448

    accuracy                           0.81      4324
   macro avg       0.78      0.79      0.79      4324
weighted avg       0.81      0.81      0.81      4324



In [13]:
df_predict = cv.transform(data_for_predict)

y_pred_proba_nb = nb_clf.predict_proba(df_predict)
max_10_index_of_nb = (-y_pred_proba_nb[:, 1]).argsort()[:10]
max_10_proba_of_nb = -np.sort(-y_pred_proba_nb[:, 1])[:10]

In [14]:
max_10_proba_of_nb

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [15]:
max_10_index_of_nb

array([49002, 45181, 50587, 45178, 53473, 45177,  7346,  7347, 29381,
        7348])

### Сравнение топ-10 токсичных текстов

#### Дерево (у дерева нет вероятностей принадлежности к классам)

In [16]:
for i in range(len(max_10_index_of_tree)):
    print('Индекс:', max_10_index_of_tree[i])
    print('Текст:', data_for_predict[max_10_index_of_tree[i]])
    print('Вероятность того, что текст содержит токсичность:', max_10_proba_of_tree[i])
    print('____________')

Индекс: 0
Текст:  анимублядский webm-треддлить приличный анимублядь и прочий аутист безграмотный быдло с дубляжом войсовер порнография и котик совет мерзкий мокрописечник вниманиеблядеть весь вид и прочее непотребство отправляться в порнотред ссылка>.длить поиск сoуса видео сохранять кадр правый клик по видео и искать он на для воспроизведение webm с 10-битный цвет нужно установить плагин vlc   и отключить встроить в браузер плеер media webm enabled=false в firefox).ый кодирование webmдоступный кодек  vp8 и vp9 для видео vorbis и opus для звук максимальный размер файл  10240кб весь файл в пост  около 40мба делать webm можно научиться в вика тред там находиться подробный информация о выбор и настройка кодековый на пример использование консольный утилит ffmpeg vpxenc и mkvmerge неочевидный момент libvorbis при указание битрейт b:a работать в режим cbr постоянный битрейт и это портить качество звук для режим vbr вместо битрейт надо указывать качество q:a параметр vbr on работать только дл

#### Наивный Байес

In [17]:
for i in range(len(max_10_index_of_nb)):
    print('Индекс:', max_10_index_of_nb[i])
    print('Текст:', data_for_predict[max_10_index_of_nb[i]])
    print('Вероятность того, что текст содержит токсичность:', max_10_proba_of_nb[i])
    print('____________')

Индекс: 49002
Текст:  безысходность и безвозвратность тхреданон очередной раз накатить волна понимание тот что время не вернуть назад что есть куча упустить возможность и шанс что никогда в жизнь большой не быть последний звонок и выпускной вечер дадада я понимать что тут большинство хиккан который в рота ебали свой одноклассник и тд и тп но я уверенный быть и благополучный класс и ты не смочь ещё раз ощутить и пройти весь заный на пример в 2017 год или 2018 потому что твой школа остаться в 90х или 00х что никогда в жизнь ты не посетить то чувство когда ты давный давно в сосничество гулял/целовал/держаться за ручка или просто пытаться это сделать с человек противоположный полый или один кома как или даже просто угнетать мысль что ты в здравый ум и память не смочь выбрать для себя другой школа выбрать спортивный секция и пожить беззаботный жизнь ты никогда не быть 17,18 год когда ты заканчивать школа и чувствовать один свобода от родитель опекун и тд ты просто осознать что весь насрать 

In [18]:
df = pd.DataFrame()
df['tree'] = max_10_index_of_tree
df['nb'] = max_10_index_of_nb
df

,tree,nb
0,0,49002
1,69096,45181
2,22685,50587
3,33521,45178
4,22687,53473
5,56703,45177
6,22689,7346
7,22684,7347
8,12830,29381
9,12827,7348


#### Результат:

Тексты совершенно не совпадают, но их всех можно считать токсичными.